![Los Angeles skyline](la_skyline.jpg)

Los Angeles, California 😎. The City of Angels. Tinseltown. The Entertainment Capital of the World! 

Known for its warm weather, palm trees, sprawling coastline, and Hollywood, along with producing some of the most iconic films and songs. However, as with any highly populated city, it isn't always glamorous and there can be a large volume of crime. That's where you can help!

You have been asked to support the Los Angeles Police Department (LAPD) by analyzing crime data to identify patterns in criminal behavior. They plan to use your insights to allocate resources effectively to tackle various crimes in different areas.

## The Data

They have provided you with a single dataset to use. A summary and preview are provided below.

It is a modified version of the original data, which is publicly available from Los Angeles Open Data.

# crimes.csv

| Column     | Description              |
|------------|--------------------------|
| `'DR_NO'` | Division of Records Number: Official file number made up of a 2-digit year, area ID, and 5 digits. |
| `'Date Rptd'` | Date reported - MM/DD/YYYY. |
| `'DATE OCC'` | Date of occurrence - MM/DD/YYYY. |
| `'TIME OCC'` | In 24-hour military time. |
| `'AREA NAME'` | The 21 Geographic Areas or Patrol Divisions are also given a name designation that references a landmark or the surrounding community that it is responsible for. For example, the 77th Street Division is located at the intersection of South Broadway and 77th Street, serving neighborhoods in South Los Angeles. |
| `'Crm Cd Desc'` | Indicates the crime committed. |
| `'Vict Age'` | Victim's age in years. |
| `'Vict Sex'` | Victim's sex: `F`: Female, `M`: Male, `X`: Unknown. |
| `'Vict Descent'` | Victim's descent:<ul><li>`A` - Other Asian</li><li>`B` - Black</li><li>`C` - Chinese</li><li>`D` - Cambodian</li><li>`F` - Filipino</li><li>`G` - Guamanian</li><li>`H` - Hispanic/Latin/Mexican</li><li>`I` - American Indian/Alaskan Native</li><li>`J` - Japanese</li><li>`K` - Korean</li><li>`L` - Laotian</li><li>`O` - Other</li><li>`P` - Pacific Islander</li><li>`S` - Samoan</li><li>`U` - Hawaiian</li><li>`V` - Vietnamese</li><li>`W` - White</li><li>`X` - Unknown</li><li>`Z` - Asian Indian</li> |
| `'Weapon Desc'` | Description of the weapon used (if applicable). |
| `'Status Desc'` | Crime status. |
| `'LOCATION'` | Street address of the crime. |

In [61]:
# Re-run this cell
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
crimes = pd.read_csv("crimes.csv", parse_dates=["Date Rptd", "DATE OCC"], dtype={"TIME OCC": str})
crimes.head()

DR_NO  Date Rptd  ...  Status Desc                                 LOCATION
0  220314085 2022-07-22  ...  Invest Cont  2500 S  SYCAMORE                     AV
1  222013040 2022-08-06  ...  Invest Cont  3300    SAN MARINO                   ST
2  220614831 2022-08-18  ...  Invest Cont                        1900    TRANSIENT
3  231207725 2023-02-27  ...  Invest Cont  6200    4TH                          AV
4  220213256 2022-07-14  ...  Invest Cont  1200 W  7TH                          ST

[5 rows x 12 columns]

# Exploratory Data Analysis

In [62]:
# Checking for duplicates
print(crimes.duplicated().sum())

0


The dataset has no duplicated row.

In [63]:
# Checking for missing values
print(crimes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185715 entries, 0 to 185714
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   DR_NO         185715 non-null  int64         
 1   Date Rptd     185715 non-null  datetime64[ns]
 2   DATE OCC      185715 non-null  datetime64[ns]
 3   TIME OCC      185715 non-null  object        
 4   AREA NAME     185715 non-null  object        
 5   Crm Cd Desc   185715 non-null  object        
 6   Vict Age      185715 non-null  int64         
 7   Vict Sex      185704 non-null  object        
 8   Vict Descent  185705 non-null  object        
 9   Weapon Desc   73502 non-null   object        
 10  Status Desc   185715 non-null  object        
 11  LOCATION      185715 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(8)
memory usage: 17.0+ MB
None


The dataset has some columns with missing values: `Vict Sex`, `Vict Descent` and `Weapon Desc`, which seems to be fine for this analysis. The information of victim's age and descent was optional, while the weapond description only applied for certain cases, so those missing values are valid.

In [64]:
# Checking for basic summary statistics in the numeric variables
print(crimes.describe())

              DR_NO       Vict Age
count  1.857150e+05  185715.000000
mean   2.255781e+08      39.999257
std    5.017438e+06      15.450227
min    2.009072e+08       2.000000
25%    2.210108e+08      28.000000
50%    2.220114e+08      37.000000
75%    2.310044e+08      50.000000
max    2.399097e+08      99.000000


The victim's age display valid values between 2 and 99. The mean victim's age in Los Angeles city is 39 +/- 15 years old.

In [65]:
# Checking for the number of categorie in the non-numeric variables
nonnum_vars = crimes.select_dtypes("object")
print([(col, crimes[col].nunique()) for col in nonnum_vars.columns])

[('TIME OCC', 1439), ('AREA NAME', 21), ('Crm Cd Desc', 104), ('Vict Sex', 4), ('Vict Descent', 20), ('Weapon Desc', 74), ('Status Desc', 6), ('LOCATION', 39286)]


We can see that the number of types of crimes committed `Crm Cd Desc` are 104, while there are 74 types of weapon used for the crimes. Thee variables could be regrouped in a deeper analysis.

## Frequency of crimes

### Frequency per hour

To get the frequency per hour, we need to obtain the hour part of the time of occurrence column, which it's a string object.

In [66]:
# Getting the hour from the time of occurrence column
crimes["TIME OCC INT"] = crimes["TIME OCC"].astype(int) # Converting the string into numeric type
hour_of_crime = np.trunc(crimes["TIME OCC INT"]/100).astype(int) # Getting only the hour
print(hour_of_crime.describe())

count    185715.000000
mean         12.813079
std           6.551456
min           0.000000
25%           8.000000
50%          13.000000
75%          18.000000
max          23.000000
Name: TIME OCC INT, dtype: float64


In [67]:
# Getting the value counts and ordering decreasingly
frequency_hours = hour_of_crime.value_counts(sort = True, ascending = False)
print(frequency_hours)

# Storing the peak crime hour
peak_crime_hour = frequency_hours.index[0]
print(peak_crime_hour)

12    13663
18    10125
17     9964
20     9579
15     9393
19     9262
16     9224
14     8872
11     8787
0      8728
21     8701
22     8531
13     8474
10     8440
8      7523
23     7419
9      7092
1      5836
6      5621
7      5403
2      4726
3      3943
4      3238
5      3171
Name: TIME OCC INT, dtype: int64
12


The most frequent hour of crimes is 12 at noon, when over 13000 crimes were reported.

### Frequency of night crimes per area

To get the night crimes we'll filter the time of occurrence (`TIME OCC INT`) between 10pm and 3:59am and then get the frequency of the areas (`AREA NAME`)

In [68]:
# Filtering the night crimes
night_crimes = crimes[(crimes["TIME OCC INT"]>=2200) | (crimes["TIME OCC INT"]<400)]
freq_area_night_crimes = night_crimes["AREA NAME"].value_counts(sort = True, ascending = False)
print(freq_area_night_crimes)

# Storing the area with most crimes reported during night time
peak_night_crime_location = freq_area_night_crimes.index[0]
print(peak_night_crime_location)

Central        3312
Hollywood      2699
77th Street    2558
Southwest      2347
Southeast      2218
Newton         2142
Olympic        2137
Pacific        1916
Rampart        1860
N Hollywood    1855
Wilshire       1716
Northeast      1684
Van Nuys       1629
West Valley    1520
Topanga        1461
Mission        1453
Harbor         1384
West LA        1369
Devonshire     1348
Hollenbeck     1315
Foothill       1260
Name: AREA NAME, dtype: int64
Central


The most frequent area for night crimes is Central, where over 3000 crimes have been reported.

### Frequency of crimes committed against victims of different age groups

To get the frequency of crimes committed against different age groups, we first need to categorize the age column and then get the count of values per category

In [69]:
# Creting the labels of age groups
age_group_labels = ["0-17", "18-25", "26-34", "35-44", "45-54", "55-64", "65+"]
# Creting the breaks for the age column
age_group_bins = [0, 17, 25, 34, 44, 54, 64, crimes["Vict Age"].max()]
# Creating the new column of age groups
crimes["Vict Age Group"] = pd.cut(crimes["Vict Age"], labels = age_group_labels, bins = age_group_bins)
# Frequency of crimes per victim's age group, making sure the order is logically correct
victim_ages = crimes["Vict Age Group"].value_counts()[["0-17", "18-25", "26-34", "35-44", "45-54", "55-64", "65+"]]
print(victim_ages)

0-17      4528
18-25    28291
26-34    47470
35-44    42157
45-54    28353
55-64    20169
65+      14747
Name: Vict Age Group, dtype: int64


The victims' age group mostly attacked has been those between 26 and 34 years old.